In [1]:
import numpy as np
import pandas as pd

# Tạo hàm lấy dữ liệu
def loadCsv(filename) -> pd.DataFrame:
    return pd.read_csv(filename)

# Tạo hàm chia dữ liệu thành train và test
def splitTrainTest(data, ratio_test):
    np.random.seed(28)  # Không thay đổi mỗi lần chạy
    index_permu = np.random.permutation(len(data))  # Xáo trộn index
    data_permu = data.iloc[index_permu]  # Lấy lại dữ liệu tương ứng với index xáo trộn
    len_test = int(len(data_permu) * ratio_test)  # Kích cỡ tập test
    test_set = data_permu.iloc[:len_test, :]  # Tập test lấy phần đầu
    train_set = data_permu.iloc[len_test:, :]  # Tập train lấy phần còn lại

    # Chia tập dữ liệu
    X_train = train_set.iloc[:, :-1]
    y_train = train_set.iloc[:, -1]
    X_test = test_set.iloc[:, :-1]
    y_test = test_set.iloc[:, -1]
    return X_train, y_train, X_test, y_test

# Hàm lấy tần số của từ
def get_words_frequency(data_X):
    bag_words = np.concatenate([i[0].split(' ') for i in data_X.values], axis=None)
    bag_words = np.unique(bag_words)  # Loại bỏ các giá trị trùng

    matrix_freq = np.zeros((len(data_X), len(bag_words)), dtype=int)  # Tạo ma trận 0
    word_freq = pd.DataFrame(matrix_freq, columns=bag_words)  # Tạo DataFrame

    for id, text in enumerate(data_X.values.reshape(-1)):
        for j in bag_words:
            word_freq.at[id, j] = text.split(' ').count(j)
    return word_freq, bag_words

# Hàm transform
def transform(data_test, bags):
    matrix_0 = np.zeros((len(data_test), len(bags)), dtype=int)
    frame_0 = pd.DataFrame(matrix_0, columns=bags)

    for id, text in enumerate(data_test.values.reshape(-1)):
        for j in bags:
            frame_0.at[id, j] = text.split(' ').count(j)
    return frame_0

# Hàm tính khoảng cách Cosine
def cosine_distance(train_X_number_arr, test_X_number_arr):
    dict_kq = dict()
    for id, arr_test in enumerate(test_X_number_arr, start=1):
        q_i = np.sqrt(sum(arr_test**2))  # Căn của tổng
        for j in train_X_number_arr:
            _tu = sum(j * arr_test)  # Tính tử
            d_j = np.sqrt(sum(j**2))  # Tính mẫu
            _mau = d_j * q_i

            kq = _tu / _mau  # Kết quả khoảng cách
            if id in dict_kq:
                dict_kq[id].append(kq)
            else:
                dict_kq[id] = [kq]

    return dict_kq

# Lớp KNNText
class KNNText:
    def __init__(self, k):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        _distance_frame = cosine_distance(self.X_train, X_test)  # Sửa ở đây: loại bỏ .values

        for i in range(len(X_test)):
            if i in _distance_frame:  # Kiểm tra xem chỉ mục có tồn tại trong _distance_frame không
                neighbors = pd.Series(_distance_frame[i]).sort_values(ascending=False).head(self.k)
                if not neighbors.empty:
                    neighbor_labels = neighbors.index.map(self.y_train)  # Lấy nhãn
                    if neighbor_labels.isnull().all():  # Kiểm tra nếu tất cả giá trị là null
                        most_common = None
                else :
                    most_common = neighbor_labels.value_counts().idxmax()  # Lấy nhãn phổ biến nhất
            else:
                most_common = None
        else:
            most_common = None 
        predictions.append(most_common)

        return predictions

# Demo bằng Education
data = loadCsv("D:\\[TH]Học máy và ứng dụng\\W3_Hoàng Đặng Bảo Trân - 2274802011053\\Education.csv")
data['Text'] = data['Text'].str.replace(',', '').str.replace('.', '')  # Loại bỏ dấu câu
X_train, y_train, X_test, y_test = splitTrainTest(data, 0.25)

# Thống kê
print(f"Số lượng X_train: {len(X_train)}")
print(f"Số lượng X_test: {len(X_test)}")

words_train_fre, bags = get_words_frequency(X_train)
print(f"Túi từ: {bags}")
print(f"Số lượng từ: {len(bags)}")
print(words_train_fre)

words_test_fre = transform(X_test, bags)

# KNN
knn = KNNText(k=2)
knn.fit(words_train_fre.values, y_train)

# Dự đoán
predictions = knn.predict(words_test_fre.values)  # Ở đây vẫn dùng .values
pred_ = pd.DataFrame(predictions, columns=['Predict'])
pred_.index = range(1, len(pred_) + 1)

# Kết quả
y_test = y_test.to_frame(name='Actual')
result = pd.concat([pred_, y_test], axis=1)
print(result)


Số lượng X_train: 39
Số lượng X_test: 13
Túi từ: ['Budget' 'Charter' 'Critical' 'Critics' 'Despite' 'Education'
 'Educational' 'Higher' 'Online' 'Parental' 'Project-based' 'Public'
 'School' 'Standardized' 'Teacher' 'The' 'Underfunded' 'a' 'academic'
 'access' 'accommodate' 'accountability' 'achievement' 'address'
 'adequate' 'adequately' 'aim' 'also' 'alternatives' 'among' 'and'
 'anxiety' 'are' 'argue' 'aspirations' 'awareness' 'backgrounds' 'based'
 'be' 'become' 'best' 'budget' 'but' 'by' 'can' 'citizenship' 'class'
 'classroom' 'classrooms' 'collaboration' 'communities' 'complex'
 'connections' 'constraints' 'contentious' 'create' 'creativity'
 'critical' 'crucial' 'cultural' 'curricula' 'curriculum' 'cuts' 'debate'
 'debated' 'decreased' 'despite' 'different' 'disparities' 'diverse'
 'divert' 'drain' 'education' 'educational' 'educators' 'effectiveness'
 'efforts' 'emphasize' 'empower' 'encourages' 'engagement' 'ensure'
 'equity' 'essential' 'evaluation' 'evaluations' 'exacerbate